# Google Cloud CMIP6 Public Data: Basic Python Example

This notebooks shows how to query the catalog and load the data using python

In [31]:
import pip
pip.main(["install","matplotlib", "pandas", "xarray", "zarr", "gcsfs", "cftime", "dask[array]", "toolz", "nc-time-axis", "openpyxl"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8915 sha256=5ce1d615eb55c9be962489267560f191b26cb3f941e04133c46668a8dc778f82
  Stored in directory: /home/jovyan/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built et-xmlfile


0

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import cftime
import dask
import toolz
import os


xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

In [3]:
plt.rcParams['figure.figsize'] = 12, 6

## Browse Catalog

The data catatalog is stored as a CSV file. Here we read it with Pandas.

In [4]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,AS-RCEC,TaiESM1,histSST,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/AS-RCEC/TaiESM1/histSST/...,NaN
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN


The columns of the dataframe correspond to the CMI6 controlled vocabulary. A beginners' guide to these terms is available in [this document](https://docs.google.com/document/d/1yUx6jr9EdedCOLd--CPdTfGDwEwzPpCF6p1jRmqx-0Q). 

Here we filter the data to find monthly surface air temperature for historical experiments.

In [5]:
df_ta = df.query("activity_id=='CMIP' & source_id == 'BCC-CSM2-MR' & table_id == '3hr' & variable_id == 'tas' & experiment_id == 'historical'")
df_ta

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
6071,CMIP,BCC,BCC-CSM2-MR,historical,r1i1p1f1,3hr,tas,gn,gs://cmip6/CMIP/BCC/BCC-CSM2-MR/historical/r1i...,NaN


Now we do further filtering to find just the models from NCAR.

## Load Data

Now we will load a single store using gcsfs, zarr, and xarray.

In [6]:
# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

# get the path to a specific zarr store (the first one from the dataframe above)
zstore = df_ta.zstore.values[-1]

# create a mutable-mapping-style interface to the store
mapper = gcs.get_mapper(zstore)

# open it using xarray and zarr
ds = xr.open_zarr(mapper, consolidated=True)
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 160, lon: 320, time: 189800)
Coordinates:
    height     float64 ...
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(160, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(320, 2), meta=np.ndarray>
  * time       (time) object 1950-01-01 00:00:00 ... 2014-12-31 21:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(47450, 1), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 dask.array<chunksize=(600, 160, 320), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          Standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  2289.0
    cmor_version:           3.3.2
    comment:                The model integration starts from the piControl e...
    contact:                Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date:          2018-11-27T03:30:12Z
    data_specs_version:     01.00.27
    description:            DECK: historical
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    external_variables:     areacella
    forcing_index:          1
    frequency:              3hrPt
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM2...
    grid:                   T106
    grid_label:             gn
    history:                2018-11-27T03:30:12Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Beijing Climate Center, Beijing 100081, China
    institution_id:         BCC
    license:                CMIP6 model data produced by BCC is licensed unde...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_source_id:       BCC-CSM2-MR
    parent_time_units:      days since 1850-01-01
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             Model described by Tongwen Wu et al. (JGR 2013; J...
    run_variant:            forcing: greenhouse gases,solar constant,aerosol,...
    source:                 BCC-CSM 2 MR (2017):   aerosol: none  atmos: BCC_...
    source_id:              BCC-CSM2-MR
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               3hr
    table_info:             Creation Date:(30 July 2018) MD5:e53ff52009d0b97d...
    title:                  BCC-CSM2-MR output prepared for CMIP6
    tracking_id:            hdl:21.14100/b880830c-7104-44d5-a02f-59bd431e816d...
    variable_id:            tas
    variant_label:          r1i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu

In [19]:
ds_sample = ds.sel(time = slice ('1950-01-01', '1950-03-01'), lon = slice(120,121), lat = slice(0,1))

In [20]:
len(ds_sample)

1

In [18]:
ds_sample

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 1, lon: 2, time: 480)
Coordinates:
    height     float64 ...
  * lat        (lat) float64 0.5607
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(1, 2), meta=np.ndarray>
  * lon        (lon) float64 120.4 121.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(2, 2), meta=np.ndarray>
  * time       (time) object 1950-01-01 00:00:00 ... 1950-03-01 21:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(480, 1), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 dask.array<chunksize=(480, 1, 2), meta=np.ndarray>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          Standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  2289.0
    cmor_version:           3.3.2
    comment:                The model integration starts from the piControl e...
    contact:                Dr. Tongwen Wu(twwu@cma.gov.cn)
    creation_date:          2018-11-27T03:30:12Z
    data_specs_version:     01.00.27
    description:            DECK: historical
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    external_variables:     areacella
    forcing_index:          1
    frequency:              3hrPt
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.BCC.BCC-CSM2...
    grid:                   T106
    grid_label:             gn
    history:                2018-11-27T03:30:12Z ; CMOR rewrote data to be co...
    initialization_index:   1
    institution:            Beijing Climate Center, Beijing 100081, China
    institution_id:         BCC
    license:                CMIP6 model data produced by BCC is licensed unde...
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_source_id:       BCC-CSM2-MR
    parent_time_units:      days since 1850-01-01
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             Model described by Tongwen Wu et al. (JGR 2013; J...
    run_variant:            forcing: greenhouse gases,solar constant,aerosol,...
    source:                 BCC-CSM 2 MR (2017):   aerosol: none  atmos: BCC_...
    source_id:              BCC-CSM2-MR
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               3hr
    table_info:             Creation Date:(30 July 2018) MD5:e53ff52009d0b97d...
    title:                  BCC-CSM2-MR output prepared for CMIP6
    tracking_id:            hdl:21.14100/b880830c-7104-44d5-a02f-59bd431e816d...
    variable_id:            tas
    variant_label:          r1i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu

In [29]:
df = ds_sample.to_dataframe().reset_index()

In [32]:
df.to_excel("test_df.xlsx")